<a href="https://colab.research.google.com/github/LJMfintech/Financial_Data_Analysis_2/blob/main/DataAnalysisHW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wrds
!pip install numpy_financial

In [ ]:
# 코드를 작성하는데 필요한 도구들 끌어오기
import wrds
import numpy_financial as npf
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
# WRDS 연결
conn = wrds.Connection()

Enter your WRDS username [root]:leeskku2025
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [ ]:
# WRDS에 있는 리스트 확인
lib_list = conn.list_libraries()
print(type(lib_list))
print(lib_list)

<class 'list'>
['aha_sample', 'ahasamp', 'audit', 'audit_audit_comp', 'audit_common', 'auditsmp', 'auditsmp_all', 'bank', 'bank_all', 'bank_premium_samp', 'banksamp', 'block', 'block_all', 'boardex_trial', 'boardsmp', 'bvd_amadeus_trial', 'bvd_bvdbankf_trial', 'bvd_orbis_trial', 'bvdsamp', 'calcbench_trial', 'calcbnch', 'candid_samp', 'cboe', 'cboe_all', 'cboe_sample', 'cboesamp', 'cddsamp', 'ciq', 'ciq_capstrct', 'ciq_common', 'ciqsamp', 'ciqsamp_capstrct', 'ciqsamp_common', 'ciqsamp_keydev', 'ciqsamp_pplintel', 'ciqsamp_ratings', 'ciqsamp_transactions', 'ciqsamp_transcripts', 'cisdmsmp', 'columnar', 'comp', 'comp_execucomp', 'comp_global_daily', 'comp_na_daily_all', 'compsamp', 'compsamp_all', 'compsamp_snapshot', 'contrib', 'contrib_as_filed_financials', 'contrib_ceo_turnover', 'contrib_char_returns', 'contrib_corporate_culture', 'contrib_general', 'contrib_global_factor', 'contrib_intangible_value', 'contrib_kpss', 'contrib_liva', 'crsp', 'crsp_a_indexes', 'crsp_a_stock', 'crsp_a_t

In [ ]:
# comp 카테고리하위 리스트 확인.
conn.list_tables(library='comp_global_daily')

['dd_group',
 'dd_group_xref',
 'dd_item',
 'dd_package',
 'g_bmiindex',
 'g_bmiindexdlynumvalue',
 'g_bmiindexidentifier',
 'g_bmiindexrel',
 'g_bmiindextradingitem',
 'g_chars',
 'g_co_aaudit',
 'g_co_adesind',
 'g_co_afnd1',
 'g_co_afnd2',
 'g_co_afnddc1',
 'g_co_afnddc2',
 'g_co_afntind1',
 'g_co_afntind2',
 'g_co_ainvval',
 'g_co_gsuppl',
 'g_co_hgic',
 'g_co_iaudit',
 'g_co_idesind',
 'g_co_ifndq',
 'g_co_ifndsa',
 'g_co_ifndytd',
 'g_co_ifntq',
 'g_co_ifntsa',
 'g_co_ifntytd',
 'g_co_industry',
 'g_co_ipcd',
 'g_co_offtitl',
 'g_company',
 'g_currency',
 'g_ecind_desc',
 'g_ecind_mth',
 'g_exrt_dly',
 'g_exrt_mth',
 'g_funda',
 'g_funda_fncd',
 'g_fundq',
 'g_fundq_fncd',
 'g_idx_daily',
 'g_idx_index',
 'g_idx_mth',
 'g_idxcst_his',
 'g_indexcst_his',
 'g_names',
 'g_names_ix',
 'g_names_ix_cst',
 'g_namesq',
 'g_sec_adesind',
 'g_sec_adjfact',
 'g_sec_afnd',
 'g_sec_afnddc',
 'g_sec_afnt',
 'g_sec_divid',
 'g_sec_dprc',
 'g_sec_dtrt',
 'g_sec_gmdivfn',
 'g_sec_gmth',
 'g_sec_g

In [ ]:
# 테이블 선정
test1 = conn.get_table(library='comp_global_daily', table='g_idx_index')
print(test1)

# test1
test1.to_csv('test1.csv', index=False)


                                            conm  gvkeyx idx13key idxcstflg  \
0                                   Jasdaq Index  115114  I192948         Y   
1                     Malta Stock Exchange Index  115118  I192956         N   
2                    Affarsvarlden General Index  150001  I000234         N   
3                     Banco Totta & Acores Index  150002  I000220         N   
4                            BCI All-Share Index  150003  I000090         N   
..                                           ...     ...      ...       ...   
730           FTSE AIM All-Share Utilities Index  277876  I220130         N   
731          FTSE AIM All-Share Technology Index  277873  I220124         N   
732  FTSE AIM All-Share Telecommunications Index  277874  I220126         N   
733    FTSE AIM All-Share Travel & Leisure Index  277875  I220128         N   
734                     ASX ALL ORDINARIES ACCUM  151333  I261044         N   

    idxstat indexcat indexgeo indexid  indextype   

In [ ]:
test2 = conn.get_table(library='comp_global_daily', table='g_idx_daily', obs = 5)
# test1
print(test2.head())
test2.to_csv('test2.csv', index=False)

   gvkeyx dvpsxd newnum oldnum  prccd prccddiv prccddivn prchd prcld  \
0  115114   <NA>   <NA>   <NA>  96.41     <NA>      <NA>  <NA>  <NA>   
1  115114   <NA>   <NA>   <NA>  79.21     <NA>      <NA>  <NA>  <NA>   
2  115114   <NA>   <NA>   <NA>  72.52     <NA>      <NA>  <NA>  <NA>   
3  115114   <NA>   <NA>   <NA>  71.61     <NA>      <NA>  <NA>  <NA>   
4  115114   <NA>   <NA>   <NA>  71.71     <NA>      <NA>  <NA>  <NA>   

     datadate  
0  1991-10-31  
1  1991-11-30  
2  1991-12-31  
3  1992-01-31  
4  1992-02-28  


In [ ]:
#
data_set = conn.raw_sql("""select gvkeyx, prccd, datadate

                          from comp_global_daily.g_idx_daily
                          where gvkeyx IN('150011')
                          """)
print(data_set.head())    # and datadate between '2020-03-01' and '2024-12-31'
print(data_set.shape)

   gvkeyx   prccd    datadate
0  150011  682.94  1985-12-31
1  150011  682.94  1986-01-01
2  150011  686.62  1986-01-02
3  150011   690.8  1986-01-03
4  150011  689.86  1986-01-06
(10404, 3)


In [ ]:
data_set.to_csv('data_set.csv', index=False)

In [ ]:
import pandas as pd

# 데이터셋 불러오기
df = pd.read_csv('data_set.csv')

# prccd 컬럼이 있는지 확인 후 수익률 계산 및 추가
def add_returns_to_df(df, price_col='prccd', return_col='return'):
    df[return_col] = df[price_col].pct_change()
    return df

df_returns = add_returns_to_df(df)
print(df_returns.head())

df_returns.to_csv('df_returns.csv', index=False)

   gvkeyx   prccd    datadate    return
0  150011  682.94  1985-12-31       NaN
1  150011  682.94  1986-01-01  0.000000
2  150011  686.62  1986-01-02  0.005388
3  150011  690.80  1986-01-03  0.006088
4  150011  689.86  1986-01-06 -0.001361


In [ ]:
from google.colab import files
uploaded = files.upload() # dwcountreturns 파일 업로드 하기

Saving dwcountryreturns.csv to dwcountryreturns.csv


In [ ]:
# 기존 파일들 불러오기
df_returns = pd.read_csv('df_returns.csv')  # data_set에서 수익률이 포함된 파일
dwcountryreturns = pd.read_csv('dwcountryreturns.csv')  # 병합할 큰 파일

# datadate 컬럼을 datetime 타입으로 변환
df_returns['datadate'] = pd.to_datetime(df_returns['datadate'])
# dwcountryreturns['datadate'] = pd.to_datetime(dwcountryreturns['datadate'])
dwcountryreturns['datadate'] = pd.to_datetime(dwcountryreturns['datadate'].astype(str), format='%Y%m%d', errors='coerce')

# datadate 기준으로 내부 조인 (inner join) 하여 병합
df_merged = pd.merge(df_returns, dwcountryreturns, on='datadate', how='inner')


# 결측치가 포함된 행 모두 삭제
df_merged_clean = df_merged.dropna()

new_order = [
    'datadate', 'country', 'fic', 'n', 'currency',
    'return', 'portret', 'portretx', 'prccd'
]

df_merged_clean = df_merged_clean[new_order]

# 결과 확인 (예: 상위 5개 행)
print(df_merged_clean.head())

df_merged_clean.to_csv('df_merged_clean.csv', index=False)

    datadate    country  fic    n currency    return   portret  portretx  \
0 1986-07-01  AUSTRALIA  AUS   54      AUD  0.005615 -0.001796 -0.001796   
1 1986-07-01    DENMARK  DNK   14      DKK  0.005615  0.012794  0.012794   
2 1986-07-01  HONG KONG  HKG   16      HKD  0.005615 -0.000580 -0.000580   
3 1986-07-01      JAPAN  JPN  638      JPY  0.005615 -0.003055 -0.003055   
4 1986-07-01  SINGAPORE  SGP   17      SGD  0.005615  0.009316  0.009316   

    prccd  
0  820.28  
1  820.28  
2  820.28  
3  820.28  
4  820.28  


In [ ]:
from google.colab import files
uploaded = files.upload() # nsshock 올리기

Saving nsshock.csv to nsshock.csv


In [27]:
import pandas as pd
import numpy as np
from datetime import timedelta

# ====================================================================
# STEP 1: 기존 데이터 불러오기
# ====================================================================
print("[Step 1] 기존 데이터 불러오기...")

# 이미 준비된 국가별 일별 수익률 데이터
df_merged_clean = pd.read_csv('df_merged_clean.csv')
df_merged_clean['date'] = pd.to_datetime(df_merged_clean['date'])

# nsshock 데이터 불러오기
nsshock = pd.read_csv('nsshock.csv')
nsshock['date'] = pd.to_datetime(nsshock['date'])

print(f"✓ df_merged_clean 데이터: {len(df_merged_clean)} 행")
print(f"  컬럼: {df_merged_clean.columns.tolist()}")
print(f"✓ nsshock 데이터: {len(nsshock)} 행")
print(nsshock.head())

# 국가별 시차 정보 (0: 당일, 1: 1일 뒤)
country_time_lag = {
    'AUSTRALIA': 1,
    'AUSTRIA': 1,
    'BELGIUM': 1,
    'BRAZIL': 0,
    'SWITZERLAND': 1,
    'CHILE': 0,
    'CHINA': 1,
    'COLOMBIA': 0,
    'GERMANY': 1,
    'DENMARK': 1,
    'EGYPT': 1,
    'SPAIN': 1,
    'FINLAND': 1,
    'FRANCE': 1,
    'UNITED KINGDOM': 1,
    'GREECE': 1,
    'HONG KONG': 1,
    'HUNGARY': 1,
    'INDONESIA': 1,
    'INDIA': 1,
    'IRELAND': 1,
    'ITALY': 1,
    'JAPAN': 1,
    'SOUTH KOREA': 1,
    'MEXICO': 0,
    'MALAYSIA': 1,
    'NETHERLANDS': 1,
    'NORWAY': 1,
    'NEW ZEALAND': 1,
    'PHILIPPINES': 1,
    'POLAND': 1,
    'PORTUGAL': 1,
    'SINGAPORE': 1,
    'SWEDEN': 1,
    'THAILAND': 1,
    'TURKEY': 1,
    'TAIWAN': 1,
    'SOUTH AFRICA': 1,
}

[Step 1] 기존 데이터 불러오기...


KeyError: 'date'

In [29]:
import pandas as pd
import numpy as np
from datetime import timedelta

# ====================================================================
# STEP 1: 기존 데이터 불러오기
# ====================================================================
print("[Step 1] 기존 데이터 불러오기...")

# 이미 준비된 국가별 일별 수익률 데이터
df_merged_clean = pd.read_csv('df_merged_clean.csv')
df_merged_clean['datadate'] = pd.to_datetime(df_merged_clean['datadate'])

# nsshock 데이터 불러오기
nsshock = pd.read_csv('nsshock.csv')
nsshock['datadate'] = pd.to_datetime(nsshock['datadate'])

print(f"✓ df_merged_clean 데이터: {len(df_merged_clean)} 행")
print(f"  컬럼: {df_merged_clean.columns.tolist()}")
print(f"✓ nsshock 데이터: {len(nsshock)} 행")
print(nsshock.head())

# ====================================================================
# STEP 2: 국가별 시차 정의 (타임존 조정)
# ====================================================================
print("\n[Step 2] 국가별 시차 정의...")

country_time_lag = {
    'AUSTRALIA': 1,
    'AUSTRIA': 1,
    'BELGIUM': 1,
    'BRAZIL': 0,
    'SWITZERLAND': 1,
    'CHILE': 0,
    'CHINA': 1,
    'COLOMBIA': 0,
    'GERMANY': 1,
    'DENMARK': 1,
    'EGYPT': 1,
    'SPAIN': 1,
    'FINLAND': 1,
    'FRANCE': 1,
    'UNITED KINGDOM': 1,
    'GREECE': 1,
    'HONG KONG': 1,
    'HUNGARY': 1,
    'INDONESIA': 1,
    'INDIA': 1,
    'IRELAND': 1,
    'ITALY': 1,
    'JAPAN': 1,
    'SOUTH KOREA': 1,
    'MEXICO': 0,
    'MALAYSIA': 1,
    'NETHERLANDS': 1,
    'NORWAY': 1,
    'NEW ZEALAND': 1,
    'PHILIPPINES': 1,
    'POLAND': 1,
    'PORTUGAL': 1,
    'SINGAPORE': 1,
    'SWEDEN': 1,
    'THAILAND': 1,
    'TURKEY': 1,
    'TAIWAN': 1,
    'SOUTH AFRICA': 1,
}

print(f"✓ 정의된 국가 수: {len(country_time_lag)}")

# ====================================================================
# STEP 3: 타임존 조정을 고려한 병합 함수
# ====================================================================
print("\n[Step 3] 병합 함수 정의...")

def merge_shock_with_time_lag(returns_df, shock_df, time_lag_map, event_window_days=5):
    """
    타임존을 고려하여 NS Shock을 수익률 데이터에 병합

    Parameters:
    -----------
    returns_df : DataFrame
        국가별 일별 수익률 데이터 (datadate, country, return, benchmark_return 등 포함)
    shock_df : DataFrame
        NS Shock 데이터 (datadate, NS_shock 포함)
    time_lag_map : dict
        국가별 타임존 오프셋 (단위: 일)
        - 0: FOMC 발표 당일이 이벤트 day 0
        - 1: FOMC 발표 다음날이 이벤트 day 0
    event_window_days : int
        이벤트 윈도우 길이 (기본 5일: day 0~4)

    Returns:
    --------
    DataFrame : NS Shock이 병합된 수익률 데이터
        컬럼: datadate, country, return, benchmark_return, NS_shock, announcement_date, event_day
    """

    merged_list = []

    # 각 FOMC 발표에 대해 반복
    for shock_idx, shock_row in shock_df.iterrows():
        announcement = shock_row['datadate']
        shock_val = shock_row['NS_shock'] if 'NS_shock' in shock_row else shock_row.iloc[1]

        # print(f"  처리중: {announcement.date()} - Shock: {shock_val:.4f}")

        # 각 국가별로 처리
        for country, lag in time_lag_map.items():
            # 해당 국가의 데이터 필터링
            country_mask = (returns_df['country'] == country)
            country_data = returns_df[country_mask].copy()

            if len(country_data) == 0:
                continue

            # 이벤트 날짜 계산 (타임존 조정)
            # lag=0: 발표 당일이 day 0
            # lag=1: 발표 다음날이 day 0
            event_date = announcement + pd.Timedelta(days=lag)

            # 각 event day (0~4)에 대해 처리
            for event_day in range(event_window_days):
                window_date = event_date + pd.Timedelta(days=event_day)

                # 해당 날짜의 거래일 찾기
                window_data = country_data[country_data['datadate'] == window_date]

                if not window_data.empty:
                    # 여러 행이 있을 경우 첫 번째 행만 선택
                    row = window_data.iloc[0].copy()
                    row['NS_shock'] = shock_val
                    row['announcement_date'] = announcement
                    row['event_day'] = event_day
                    merged_list.append(row)

    if merged_list:
        return pd.DataFrame(merged_list)
    else:
        return pd.DataFrame()

print("✓ 병합 함수 정의 완료")

# ====================================================================
# STEP 4: 병합 실행
# ====================================================================
print("\n[Step 4] 데이터 병합 실행 (이 과정은 시간이 걸릴 수 있습니다)...")

df_final = merge_shock_with_time_lag(
    df_merged_clean,
    nsshock,
    country_time_lag,
    event_window_days=5
)

print(f"✓ 병합 완료: {len(df_final)} 행")

# ====================================================================
# STEP 5: 최종 데이터 정리
# ====================================================================
print("\n[Step 5] 최종 데이터 정리...")

# 컬럼 재정렬
required_columns = ['announcement_date', 'datadate', 'event_day', 'country',
                    'return', 'NS_shock']

# df_merged_clean에 있는 모든 컬럼 보존
available_cols = [col for col in required_columns if col in df_final.columns]
other_cols = [col for col in df_final.columns if col not in required_columns]

# 최종 컬럼 순서
final_order = available_cols + other_cols
df_final = df_final[final_order]

# 타임스탬프 기준 정렬
df_final = df_final.sort_values(['announcement_date', 'country', 'event_day']).reset_index(drop=True)

print(f"✓ 최종 컬럼 순서:")
print(df_final.columns.tolist())

# ====================================================================
# STEP 6: 데이터 확인 및 통계
# ====================================================================
print("\n[Step 6] 데이터 확인...")

print(f"\n=== 최종 데이터셋 통계 ===")
print(f"전체 관측치: {len(df_final)}")
print(f"FOMC 발표 횟수: {df_final['announcement_date'].nunique()}")
print(f"포함된 국가 수: {df_final['country'].nunique()}")

print(f"\n국가별 관측치 분포:")
print(df_final['country'].value_counts().head(15))

print(f"\n이벤트 윈도우별 분포:")
print(df_final['event_day'].value_counts().sort_index())

print(f"\n샘플 데이터 (첫 15행):")
print(df_final.head(15).to_string())

# ====================================================================
# STEP 7: 데이터 저장
# ====================================================================
print("\n[Step 7] 데이터 저장...")

output_file = 'problem1_final_merged_data.csv'
df_final.to_csv(output_file, index=False)

print(f"✓ 데이터 저장 완료: {output_file}")
print(f"  파일 크기: {len(df_final)} 행, {len(df_final.columns)} 열")

# ====================================================================
# STEP 8: 데이터 품질 확인
# ====================================================================
print("\n[Step 8] 데이터 품질 확인...")

print(f"\n결측값 확인:")
print(df_final.isnull().sum())

print(f"\n수익률(return) 기초통계:")
print(df_final['return'].describe())

print(f"\nNS_shock 기초통계:")
print(df_final['NS_shock'].describe())

# 저장된 파일 검증
print(f"\n[최종 확인] 저장된 파일 검증...")
df_check = pd.read_csv(output_file)
print(f"✓ 저장 파일 확인: {len(df_check)} 행, {len(df_check.columns)} 열")
print(f"✓ 컬럼명: {df_check.columns.tolist()}")

print(f"\n{'='*70}")
print(f"✓✓✓ PROBLEM 1 완성! ✓✓✓")
print(f"{'='*70}")
print(f"\n다음 단계:")
print(f"- 이 데이터를 Problem 2(Event Study Design)에서 사용하세요")
print(f"- 각 국가별로 비정상수익률(Abnormal Return) 계산")
print(f"- 누적 비정상수익률(CAR) 계산")

[Step 1] 기존 데이터 불러오기...
✓ df_merged_clean 데이터: 289503 행
  컬럼: ['datadate', 'country', 'fic', 'n', 'currency', 'return', 'portret', 'portretx', 'prccd']
✓ nsshock 데이터: 234 행
    datadate  GSS_target  GSS_path      NS
0 1995-02-01      0.0597    0.0268  0.0419
1 1995-03-28      0.0258    0.0568  0.0288
2 1995-05-23      0.0040   -0.0064  0.0009
3 1995-07-06     -0.1033   -0.3785 -0.1510
4 1995-08-22      0.0508    0.0217  0.0354

[Step 2] 국가별 시차 정의...
✓ 정의된 국가 수: 38

[Step 3] 병합 함수 정의...
✓ 병합 함수 정의 완료

[Step 4] 데이터 병합 실행 (이 과정은 시간이 걸릴 수 있습니다)...
✓ 병합 완료: 23222 행

[Step 5] 최종 데이터 정리...
✓ 최종 컬럼 순서:
['announcement_date', 'datadate', 'event_day', 'country', 'return', 'NS_shock', 'fic', 'n', 'currency', 'portret', 'portretx', 'prccd']

[Step 6] 데이터 확인...

=== 최종 데이터셋 통계 ===
전체 관측치: 23222
FOMC 발표 횟수: 234
포함된 국가 수: 38

국가별 관측치 분포:
country
MEXICO            732
BRAZIL            728
AUSTRALIA         691
UNITED KINGDOM    681
NEW ZEALAND       677
SWITZERLAND       675
NORWAY            675
SWED